In [1]:
from xbbg import blp
from xbbg.core import conn,process
from datetime import date
from datetime import datetime

import pandas as pd

In [2]:
SAVEDATA = True
DATE_HISTORIC = '2022-10-25'

# STIR Futures

In [3]:
def get_futures_chain(fut_active,Nchain):
    chain = blp.bds(fut_active,'FUT_CHAIN')
    futflds = ['PX_SETTLE_LAST_DT','PX_LAST','OPEN_INT','FUT_DAYS_EXPIRE','LAST_TRADEABLE_DT']
    ticks = chain['security_description'].iloc[:Nchain]
    futdata = blp.bdp(ticks, futflds).sort_values('last_tradeable_dt')
    return futdata

### Get Live Data

In [4]:
fut = 'FFA Comdty'
Nchain = 18
ffdata = get_futures_chain(fut,Nchain)
ffdata.index.name = 'ticker'

DATE = ffdata['px_settle_last_dt'].iloc[0].strftime('%Y-%m-%d')

### Get Historic Date

In [5]:
futflds = ['PX_LAST']
chain = blp.bds(fut,'FUT_CHAIN')
ticks = chain['security_description'].iloc[:Nchain]
futdata_historic = blp.bdh(ticks, futflds, start_date=DATE_HISTORIC).droplevel(level=1,axis=1).iloc[[0],:].T

### Fed Funds

In [6]:
fedfunds = blp.bdp(['FEDL01 Index', 'FDTR Index'],['PX_LAST','SHORT_NAME'],single_date_override=DATE)
fedfunds['date'] = DATE
fedfunds.index.name = 'ticker'
fedfunds

### Fed Calendar

In [7]:
feddts = ['2023-05-03',
        '2023-06-14',
        '2023-07-26',
        '2023-09-20',
        '2023-11-01',
        '2023-12-13',
         '2024-01-26',
         '2024-03-22',
         '2024-05-03',
         '2024-06-14',
         '2024-07-26']

feddts = pd.Series([datetime.strptime(dt,'%Y-%m-%d').date() for dt in feddts], name='meeting dates')

***

## SOFR Futures

In [8]:
fut = 'SERA Comdty'
Nchain = 12
sofrdata1 = get_futures_chain(fut,Nchain)
sofrdata1

In [9]:
fut = 'SFRA Comdty'
Nchain = 12
sofrdata3 = get_futures_chain(fut,Nchain)
sofrdata3

### Save Data

In [10]:
if SAVEDATA:    
    outfile = f'../data/fedfutures_{DATE}.xlsx'
    with pd.ExcelWriter(outfile) as writer:  
        ffdata.to_excel(writer, sheet_name= 'fed futures', index=True) 
        fedfunds.to_excel(writer, sheet_name= 'fed funds', index=True) 
        feddts.to_excel(writer, sheet_name= 'fed meetings', index=False)
        futdata_historic.to_excel(writer, sheet_name= f'fed futures {DATE_HISTORIC}', index=True)